# Recommender Systems 2024/25

### Practice 7 – Graph-based Models

## Graph-based Recommenders

### What is a Graph?

In layman terms, a graph is a set of **interconnected entities**. Each entity is represented by a *node*, while the connections are called *edges*. Thus, mathematically speaking, a graph $G \in \mathbb{R}^{n\times n}$ can be represented as two sets, $N$ (nodes) and $E$ (edges).$$G = (N,E) \\ N = \{x_i | i \in [1, |N|]_{\mathbb{N}}\} \\ E = \{\{x_i, x_j\} | x_i,x_j \in N\}$$

### Graph Notation

#### Graph Chart

Nodes are usually represented graphically as circles, with lines (=edges) connecting one another

[GIF Here]

#### Adjacency Lists

For each node in the graph, its adjacency list contains all nodes it is connected to. Pros: easily readable, cons: redundant

```python
G = {
    "n1" : ["n2", "n3", "n5"],
    "n2" : ["n1", "n4"],
    "n3" : ["n1", "n5"],
    "n4" : ["n2"],
    "n5" : ["n1", "n3"]
}
```

#### Adjacency Matrices

##### Node-2-Edge

...

##### Node-2-Node

...

- Part 1: generic structure of a graph
    - Nodes
    - Edges
    - (E.g. : Gif of my knowledge graph on Obsidian)
    - Notation: chart OR adjacency matrix OR adjacency lists
        - Adjacency matrices: node2node OR node2edge
            - For our purpose, we use node2node adjacency matrices
- Part 2: graphs in recommendation
    - Nodes: users, items
    - Edges: interactions
    - Bipartite graph: we can distinguish 2 classes of nodes, with edges only connecting nodes of class 1 to nodes of class 2 and vice-versa. No inter-class connections
        - Adjacency matrix of a graph of interactions: URM!
            - `G = [[[0][URM]], [[URM*], [0]]]`
    - Path: sequence of connected nodes that starts with node A and ends with node B
        - Recommendation idea: we can use paths as collaborative signals
- Part 3: random walks
    - Recap on mathematics (from textbook)
    - Illustrative example (from slides)
- Part 4: $P^3$, $P^3_\alpha$, $RP^3_\beta$
    - $P^3$: 3-step random walks
        - Theory recap (from textbook, don't explain)
        - Illustrative example (from slides)
    - $P^3_\alpha$: add an exponent
        - Theory recap (from textbook, don't explain)
        - Illustrative example (from slides)
    - $RP^3_\beta$: ...
        - Theory recap (from textbook, don't explain)
        - How to use (from framework), strong baseline! Good for hybrids
- Part 5: Adding item metadata, tripartite graph
    - Theory recap (from slides)
    - Illustrative example with $P^3$